# NDN-DPDK PCI/XDP Comparison Test

First, we grab fablib manager and setup some imports we'll need later.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
import json
import shlex
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

We then find a site with the correct number of resources. You may rerun this cell if you don't get a site you like (i.e. something in a different country)

In [ ]:
# find a site with appropriate cores
cores_per_node = 24 # this should be at least 16
ram_per_node = 32
disk_per_node = 20
site1 = fablib.get_random_site(
    filter_function=lambda x: x["cores_available"] > cores_per_node * 3 and x["ram_available"] > ram_per_node * 3 and x["disk_available"] > disk_per_node * 3 and x["nic_connectx_6_available"] > 2
)
producer_name = f"{site1.lower()}-p1-a"
hosts = [f"{site1.lower()}-c1-a", f"{site1.lower()}-r1-a", producer_name]
fablib.show_site(site1, 'json')

Name the slice here, then create the slice provisioning request in the next cell.

In [ ]:
slice_name = 'ndn-poc'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

for h in hosts:

    node = slice.add_node(name=h, cores=cores_per_node, ram=ram_per_node, disk=disk_per_node, site=site1, image='default_ubuntu_22')
    if "p1" in h:
        node.add_component(model="NVME_P4510", name="disk")
    node.add_fabnet(nic_type='NIC_ConnectX_6')
    node.add_post_boot_upload_directory("scripts")
    node.add_post_boot_execute("./scripts/install_dependencies.sh")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")

#Submit Slice Request
slice.submit();

Configure the NVME drive for the producer node

In [ ]:
fs_path = '/srv/fileserver'
slice.get_node(name=producer_name).get_component('disk').configure_nvme(mount_point=fs_path)

Set up CPU Isolation to pin certain cores to NDN-DPDK

In [ ]:
# CPU Isolation
execute_threads = {}
for node in slice.get_nodes():
    manager_dir = '/etc/systemd/system.conf.d'
    service_dir = '/etc/systemd/system/ndndpdk-svc@127.0.0.1\:3030.service.d'
    start = 0
    reserved = 6
    end = cores_per_node - 1
    if node.get_name() != producer_name:
        execute_threads[node] = node.execute_thread(f'''
            sudo mkdir -p {manager_dir}
            echo "[Manager]\nCPUAffinity={start}-{reserved - 1}" | sudo tee {manager_dir}/cpuset.conf
            sudo mkdir -p {service_dir}
            echo "[Service]\nCPUAffinity={reserved}-{end}" | sudo tee {service_dir}/cpuset.conf
            sudo systemctl reboot
        ''')
    else:
        fs_dir = '/etc/systemd/system/ndndpdk-svc@127.0.0.1\:3031.service.d'
        mid = (end - reserved) // 2 + reserved - 1
        execute_threads[node] = node.execute_thread(f'''
            sudo mkdir -p {manager_dir}
            echo "[Manager]\nCPUAffinity={start}-{reserved - 1}" | sudo tee {manager_dir}/cpuset.conf
            sudo mkdir -p {service_dir}
            echo "[Service]\nCPUAffinity={reserved}-{mid}" | sudo tee {service_dir}/cpuset.conf
            sudo mkdir -p {fs_dir}
            echo "[Service]\nCPUAffinity={mid + 1}-{end}" | sudo tee {fs_dir}/cpuset.conf
        ''')
for thread in execute_threads.values():
    thread.result()
for node in slice.get_nodes():
    node.os_reboot()
slice.wait_ssh(progress=True)
slice.post_boot_config()

Here are all of the settings for the various NDN services that need to be booted up.

In [ ]:
FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}
FS_ACTIVATE = {
    'eal': {
        'coresPerNuma': { '0': 4 },
        'memPerNuma': {'0': 4*1024},
        'filePrefix': 'producer',
    },
    'mempool': {
        'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**16-1},
        'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
    },
    'face': {
        'scheme': 'memif',
        'socketName': MEMIF_ARGS['socketName'],
        'id': MEMIF_ARGS['id'],
        'dataroom': 9000,
        'role': 'client',
    },
    'fileServer': {
        'mounts': [
            {'prefix': "/producer/files", 'path': "/srv/fileserver"}
        ],
        'segmentLen': 6 * 1024,
        'uringCapacity': 4096,
    },
}

adj_list = {
    hosts[0]: [hosts[1]],
    hosts[1]: [hosts[0], hosts[2]],
    hosts[2]: [hosts[1]],
}

forwarder_hashes = {}
face_hashes = {}

## PCI NDN Setup
This will run through set up for NDN-DPDK with PCI drivers

In [ ]:
for n in slice.get_nodes():
    n.os_reboot()
slice.wait_ssh(progress=True)

# setup hugepages for NDN-DPDK
# we run this again because the nodes restarted after we made CPU isolation changes

for h in hosts:
    node = slice.get_node(h)
    output = node.execute("sudo dpdk-hugepages.py --pagesize 1G --setup 10G")

In [ ]:
# add eth ports to each node
for h in hosts:
    node = slice.get_node(h)
    output = node.execute(f'''
        sudo ndndpdk-ctrl systemd restart
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --pci 07:00.0 --mtu 9000 --rx-flow 16
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

# add interface between nodes in adj_list
for h in adj_list:
    local_mac = forwarder_hashes[h]['macAddr']
    node = slice.get_node(h)
    for remote in adj_list[h]:
        remote_mac = forwarder_hashes[remote]['macAddr']
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
        face_hashes[f"{h}:{remote}"] = json.loads(output[0])

In [ ]:
# add fileserver configuration to producer
producer_node = slice.get_node(producer_name)
output = producer_node.execute(f'''
    MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
    ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh $(echo $MEMIF_FACE | jq -r .id)
    ''')
# start fileserver instance
output = producer_node.execute(f'''
    sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
    echo {shlex.quote(json.dumps(FS_ACTIVATE))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
    ''')

In [ ]:
# add FIB entries to point to producer
c_node = slice.get_node(hosts[0])
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
output = c_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {c_to_r['id']}")

r_node = slice.get_node(hosts[1])
r_to_p = face_hashes[f"{hosts[1]}:{hosts[2]}"]
r_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {r_to_p['id']}")

In [ ]:
# setup test files on producer
# you can edit the "head" commands to produce whatever size file you want
# we copy each file 5 times in order to produce different filenames so we don't accidentally cache results in the content store
output = producer_node.execute(f'''
cd /srv/fileserver
sudo head -c 500K </dev/urandom >500Ktestfile
sudo cp 500Ktestfile 500Ktestfile2
sudo cp 500Ktestfile 500Ktestfile3
sudo cp 500Ktestfile 500Ktestfile4
sudo cp 500Ktestfile 500Ktestfile5
sudo head -c 100M </dev/urandom >100Mtestfile
sudo cp 100Mtestfile 100Mtestfile2
sudo cp 100Mtestfile 100Mtestfile3
sudo cp 100Mtestfile 100Mtestfile4
sudo cp 100Mtestfile 100Mtestfile5
''')

Run your tests on the consumer by ssh-ing into it and running the following line (modifying the "--name" command line for each file): 
```
sudo ndndpdk-godemo fetch --filename /tmp/500 --name /producer/files/500Ktestfile
```

## XDP tests

Once the PCI tests are done, now we reprovision NDN-DPDK with XDP drivers (note, you may need to change the netif parameter if your slice provisions with a different network interface name):

In [ ]:
# reboot each node to reset devices
for h in hosts:
    node = slice.get_node(h)
    node.os_reboot()
slice.wait_ssh(progress=True)

# add eth ports to each node
for h in hosts:
    print("Running on host: " + h)
    node = slice.get_node(h)
    output = node.execute(f'''
        sudo dpdk-hugepages.py --pagesize 1G --setup 10G
        sudo ndndpdk-ctrl systemd start
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif enp7s0np0 --mtu 1500 --xdp
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

# add interface between nodes in adj_list
for h in adj_list:
    local_mac = forwarder_hashes[h]['macAddr']
    node = slice.get_node(h)
    for remote in adj_list[h]:
        remote_mac = forwarder_hashes[remote]['macAddr']
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
        face_hashes[f"{h}:{remote}"] = json.loads(output[0])
        
# add fileserver configuration to producer
producer_node = slice.get_node(producer_name)
output = producer_node.execute(f'''
    MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
    ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh $(echo $MEMIF_FACE | jq -r .id)
    ''')

# start fileserver instance
output = producer_node.execute(f'''
    sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
    echo {shlex.quote(json.dumps(FS_ACTIVATE))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
    ''')

# add FIB entries to point to producer
c_node = slice.get_node(hosts[0])
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
output = c_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {c_to_r['id']}")

r_node = slice.get_node(hosts[1])
r_to_p = face_hashes[f"{hosts[1]}:{hosts[2]}"]
r_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {r_to_p['id']}")

Now you can run the same tests as before on the consumer:
```
sudo ndndpdk-godemo fetch --filename /tmp/500 --name /producer/files/500Ktestfile
```

## Slice extention/deletion
If you want, you can extend the slice with this cell:

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    slice = fablib.get_slice(slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")
slice.update()
slice.show()

Once you are done with the slice, make sure you delete it!

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()

In [ ]:
# wait a few seconds before running this one to confirm deletion
slice.update()
slice.show()